# Planet to planet low-thrust

In this tutorial we show the use of the {class}`pykep.trajopt.direct_pl2pl` to find a low-thrust (zero-order hold continous thrust) trajectory connecting two moving planets. 

The decision vector for this class, compatible with pygmo {cite:p}`pagmo` UDPs (User Defined Problems), is:

$$
\mathbf x = [t_0, m_f, V_{sx}^\infty, V^\infty_{sy}, V^\infty_{sz}, V^\infty_{fx}, V^\infty_{fy}, V^\infty_{fz}, u_{x0}, u_{y0}, u_{z0}, u_{x1}, u_{y1}, u_{z1}, ..., T_{tof}]
$$

containing the starting epoch $t_0$ as a MJD2000, the final mass $m_f$ as well as the starting and final $V^{\infty}$, throttles and the time-of-flight $T_{tof}$.

:::{note}
This notebook makes use of the commercial solver SNOPT 7 and to run needs a valid `snopt_7_c` library installed in the system. In case SNOPT7 is not available, you can still run the notebook using, for example `uda = pg.algorithm.nlopt("slsqp")` with minor modifications.

Basic imports:

In [26]:
import pykep as pk
import numpy as np
import time
import pygmo as pg
import pygmo_plugins_nonfree as ppnf
import time

from matplotlib import pyplot as plt

We start defining the problem data. For the purpose of this simple notebook we choose a simple Earth to Mars transfer.

In [27]:
# Problem data

####
## Testcase Earth-Mars with
mu = pk.MU_SUN
max_thrust = 0.3
isp = 3000
veff = isp * pk.G0
tof = 550.0

posvel0 = [
    [-125036811000.422, -83670919168.87277, 2610252.8064399767],
    [16081.829029183446, -24868.923007449284, 0.7758272135425942]
]
posvelf = [
    [-169327023332.1986, -161931354587.78766, 763967345.9733696],
    [17656.297796509956, -15438.116653052988, -756.9165272457421]
]

# Define initial and target
p1 = pk.planet(pk.udpla.keplerian(when=pk.epoch(0), posvel = posvel0, mu_central_body=mu))
p2 = pk.planet(pk.udpla.keplerian(when=pk.epoch(tof), posvel = posvelf, mu_central_body=mu))

# Initial state
ms = 1500.0

# Number of segments
nseg = 4


In [28]:
# # Normalise
# L = pk.AU
# TIME = np.sqrt(L**3 / mu)
# VEL = L / TIME
# ACC = VEL / TIME
# MASS = ms

# print(f'Normalise L {L:.4f} T {TIME:.4f} V {VEL:.4f} M {MASS:.4f}')

# # Problem data
# mu = mu / (L**3 / TIME**2)
# max_thrust = max_thrust / (MASS * L / TIME**2)
# veff = veff / VEL

# # Initial state
# ms = ms / MASS
# posvel0[0] = (np.array(posvel0[0]) / L).tolist()
# posvel0[1] = (np.array(posvel0[1]) / VEL).tolist()

# # Final state
# posvelf[0] = (np.array(posvelf[0]) / L).tolist()
# posvelf[1] = (np.array(posvelf[1]) / VEL).tolist()

# # tof
# tof = tof / TIME

We here instantiate two different versions of the same UDP (User Defined Problem), with analytical gradients and without. 

For the purpose of this simple notebook we choose a relatively simple Earth to Mars transfer with an initial $V_{\infty}$ of 3 km/s.

In [29]:
udp_g = pk.trajopt.direct_pl2pl(
        pls=p1,
        plf=p2,
        ms=ms,
        mu=mu,
        max_thrust=max_thrust,
        veff=veff,
        t0_bounds=[0.0, 0.0],
        tof_bounds=[tof,tof],
        mf_bounds=[ms*0.5, ms],
        vinfs=0.,
        vinff=0.,
        nseg=nseg,
        cut=0.6,
        mass_scaling=ms,
        r_scaling=pk.AU,
        v_scaling=pk.EARTH_VELOCITY,
        with_gradient=False,
        high_fidelity=False
)

## Analytical performances of the analytical gradient

In [30]:
# Check gradientss
# Analytical
an_grad = udp_g.gradient(pop_g.champion_x)
# Numerical
num_grad = pg.estimate_gradient_h(udp_g.fitness, pop_g.champion_x, dx=1e-8)

# Suppose sparsity gives the positions (flat indices in num_grad)
sparsity = udp_g.gradient_sparsity()  # e.g. [0, 3, 10, ...]

# Make an empty dense gradient
dim = int(9+3*nseg)
dense_ana_grad = np.zeros_like(num_grad).reshape(int(len(num_grad)/dim),dim)
num_grad = num_grad.reshape(int(len(num_grad)/dim),dim)

# Fill in analytical entries at the right positions
for jj in range(len(an_grad)):
    dense_ana_grad[sparsity[jj][0], sparsity[jj][1]] = an_grad[jj]
    diff_tmp = abs(dense_ana_grad[sparsity[jj][0], sparsity[jj][1]] - num_grad[sparsity[jj][0], sparsity[jj][1]])
    if diff_tmp > 1e-3:
        print('sparsity[jj]',sparsity[jj] , 'analytical', dense_ana_grad[sparsity[jj][0], sparsity[jj][1]],'numerical', num_grad[sparsity[jj][0], sparsity[jj][1]],'diff',diff_tmp)
# dense_ana_grad = dense_ana_grad.reshape(-1)
# num_grad = num_grad.reshape(-1)

# Now you can compare
diff_bool = np.allclose(num_grad, dense_ana_grad, atol=1e-3, rtol=1e-3)
diff = num_grad - dense_ana_grad
# print('Analytical', dense_ana_grad)
# print('Numerical', num_grad)
print('')
print('diff_bool', diff_bool)
print('len J', 10 + nseg)
print('len x', 9 + 3*nseg)
print('shape(diff)', np.shape(diff))
print("‖diff‖:", np.linalg.norm(diff))


diff_bool True
len J 14
len x 21
shape(diff) (14, 21)
‖diff‖: 5.61561005875e-07


First the analytical gradient:

In [ ]:
%%timeit
udp_g.gradient(pop_g.champion_x)

Then a simple numerical gradient based on finite differences:

In [ ]:
%%timeit
pg.estimate_gradient(udp_g.fitness, pop_g.champion_x)

Then a higher order numerical gradient:

In [ ]:
%%timeit
pg.estimate_gradient_h(udp_g.fitness, pop_g.champion_x)

The analytical gradient its exact and faster, seems like a no brainer to use it. 

In reality, the effects on the optimization technique used are not straightforward, making the option to use numerical gradients still interesting in some, albeit rare, cases.

## Solving the low-thrust transfer

We define (again) the optimization problem, and set a tolerance for *pagmo* to be able to judge the relative value of two individuals. 

:::{note}
This tolerance has a different role from the numerical tolerance set in the particular algorithm chosen to solve the problem and is only used by the *pagmo* machinery to decide outside the optimizer whether the new proposed indivdual is better than what was the previous *champion*.

In [ ]:
prob_g = pg.problem(udp_g)
prob_g.c_tol = 1e-6

... and we define an optimization algorithm.

In [ ]:
snopt72 = "/Users/dario.izzo/opt/libsnopt7_c.dylib"
uda = ppnf.snopt7(library=snopt72, minor_version=2, screen_output=False)
uda.set_integer_option("Major iterations limit", 2000)
uda.set_integer_option("Iterations limit", 20000)
uda.set_numeric_option("Major optimality tolerance", 1e-3)
uda.set_numeric_option("Major feasibility tolerance", 1e-11)

algo = pg.algorithm(uda)

In [ ]:
# uda = pg.nlopt("slsqp")
# algo = pg.algorithm(uda)

In [ ]:
# ip = pg.ipopt()
# ip.set_numeric_option("tol", 1E-9) # Change the relative convergence tolerance
# ip.set_integer_option("max_iter", 500) # Change the maximum iterations
# ip.set_integer_option("print_level", 0) # Makes Ipopt unverbose
# ip.set_string_option("nlp_scaling_method", "none") # Removes any scaling made in auto mode
# ip.set_string_option("mu_strategy", "adaptive") # Alternative is to tune the initial mu value
# algo = pg.algorithm(ip)

We solve the problem from random initial guess ten times and only save the result if a feasible solution is found (as defined by the criterias above)

In [ ]:
masses = []
xs = []
for i in range(10):
    pop_g = pg.population(prob_g, 1)
    pop_g = algo.evolve(pop_g)
    if(prob_g.feasibility_f(pop_g.champion_f)):
        print(".", end="")
        masses.append(pop_g.champion_x[1])
        xs.append(pop_g.champion_x)
        break
    else:
        print("x", end ="")
print("\nBest mass is: ", np.max(masses))
print("Worst mass is: ", np.min(masses))
best_idx = np.argmax(masses)

And we plot the trajectory found:

In [ ]:
udp_g.pretty(xs[best_idx])

In [ ]:
ax = udp_g.plot(xs[best_idx], show_gridpoints=True)
ax.view_init(90, 0)